# CHAPTER3 Nuxt.jsの機能の活用

この章では、レイアウト、ミドルウェア、プラグイン、Vuexのモジュールモードについて学ぶ

## 3.19 layoutディレクトリによる、レイアウトの共通化

### レイアウトの構築機能について

フッターや、グローバルナビゲーションなど、ページに共通して表示されるコンポーネントを`layouts`ディレクトリに配置することで、共通化できる.

これを`レイアウトの共通化`という。

### レイアウトのルールとdefault.vue　の編集
Nuxt.jsではデフォルトのレイアウトとして、`layouts/default.vue`が用意されている。
特に指定のない場合は、このレイアウトが適用される。

### 複数レイアウトの管理

カテゴリ分けをして、それぞれのページに適応するレイアウトを設定することもできる。

ページにレイアウトを指定するには、`layout`プロパティを使用し、レイアウトのVueファイル名を指定する。

```js
export default {
  layout: 'default' //デフォルト値
  layout: 'other' //layouts/other.vueを使用する
}
```

### 今見ているレイアウトは、VueDevToolsで確認できる

現在のレイアウトは、VueDevTools上でレイアウトの名前を確認できる。

### レイアウト設計のベストプラクティス

* `default.vue`は標準で適用されるレイアウト。従って、LPやトップページなど他のページと大きく異なるデザインのページのレイアウトには使用しない方が良い。
    * レイアウト適用漏れや、修正範囲の拡大などを防ぐため
* アプリケージョン全体で使用するレイアウト：`default.vue`
* 他と大きく異なるページ：個別でレイアウトを設計する




## 3.20 Nuxt.jsのライフサイクル

Nuxt.jsには、本来のVue.jsのライフサイクルに加え、Nuxt.js独自のライフサイクルが存在する。

* Incoming Request
    * ユーザーからの要求
* nuxtServerInit
    * クライアントからの要求直後に実行されるアクション
    * VuexストアのルートストアにnuxtServerInitアクションがある場合、それが実行される。
    * アプリケーションのストア初期化処理を記述することができる。
    * 必ず入れておきたいデータやプロパティなどをここで格納することができる。
* middleware()
    * 全体で機能させる単一特定の機能などを提供する
* validate()
    * ページコンポーネントのvalidateメソッドを実行する
* asyncData() & fetch()
    * asyncData()とfetch()の実行
* Render
    * DOMのレンダリング
    * Vue.jsのライフサイクルの実行


# 3.21 middlewareによるグローバルなフックの登録

middlewareは、インデックスページも含めた各ページの初期化前に実行される。
SSRの処理が行われる前に実行され、ユーザー承認処理やページリダイレクトなどの処理を行うことができる。

### middlewareを利用した認証のあるルーティングの実装

ここでは、以下の3つのページに関して、アクセス前に認証を行う。
|アドレス|権限|
|:--|:--|
|/|全ユーザーに開放。特に認証なし。|
|/login|未ログインユーザーのみ、ログインしている場合は`/`へ飛ぶ|
|/authed-route|既ログインユーザーのみ。ログインしていない場合は`login`へ飛ぶ|

なお今回はCookieを使用して、ログイン状態を管理するので、`Universal-Cookie`をインストールする。

```bash
npm install universal-cookie
```

以下3つのページコンポーネントを作成する。
* `pages/index.vue`
```vue
<template>
  <div>
    <h2>Index Page</h2>
    <ul>
      <li><nuxt-link to="/login">to login page</nuxt-link></li>
      <li><nuxt-link to="/authed-route">to require authorization page</nuxt-link></li>
    </ul>
  </div>
</template>
```
* `pages/login.vue`
```vue
<template>
    <div>
        <h2>login page</h2>
        <p>
            <button type="button" @click="login">ログイン</button>
            <br>
            <nuxt-link to="/">return to top</nuxt-link>
        </p>
    </div>
</template>

<script>
import Cookies from "universal-cookie";

export default {
    methods:{
        login() {
            const  cookies = new Cookies();
            cookies.set('credential', true, { maxAge: 90 });
            this.$router.push('/');
        }
    }
}
</script>

```

* `pages/authed-route.vue`
```vue
<template>
    <div>
        <h2>require authorization</h2>
        <p>Cookieの`credential`の値がTrueの時のみアクセス可</p>
        <br>
        <nuxt-link to="/">return to top</nuxt-link>
    </div>
</template>

<script lang="js">

</script>
```


### ミドルウェアの作成とグローバルへの登録

`middleware`ディレクトリを作成し、その配下にjsファイルを作成する。

* `middleware/auth.js`
```js
export default () => {
    if(process.browser) {
        cosole.log('on browser');
    } else {
        console.log('on server');
    }
}
```

ミドルウェアは`default export` により、唯一の関数を返す必要がある。

今回は、各ページへアクセスした時点で、middlewareが実行され、規定のコンソールメッセージが表示される。

実際にミドルウェアを登録するには、`nuxt.config.js`に以下のように記述する。

```js
module.exports = {
    // ...
    router: {
        middleware: 'auth'
    }
}
```

### ミドルウェアへの認証の実装

各ミドルウェアの関数の引数には、`context`と呼ばれるオブジェクトが規定で渡される。
* ページのリダイレクト、現在のページURLの参照などの機能を提供する
* contextについては、Nuxt.jsのドキュメントを参照すること。

ミドルウェアへの実装は、先に作成した`middleware/jsファイル`に記述する。

```js
// middleware/auth.js
import Cookies from "universal-cookie";

export default (context) => {
    /**
     * context: {
     *  req,
     *  route,
     *  redirect, //302リダイレクトの機能を提供する関数
     * }
     */
    console.log(context.route.path);
    if(['/'].includes(context.route.path)) {
        return;
    }
    const cookies = req ? new Cookies(req.headers.cookie) : new Cookies();
    const credential = cookies.set('credential');

    if(credential && context.route.path === '/login') {
        context.redirect('/');
    } else if(!credential && context.route.path !== '/login') {
        context.redirect('/login');
    }
}
```


## 3.22 プラグインによるVue.jsプラグイン資産の有効活用

アプリケーション全体で使うようなら外部ライブラリ、CDNなどを`utils`など独自領域を作り共有化することも不可能ではないが、Nuxt.jsでは、プラグインとして作成することで、以下２つのメリットがある。
* Nuxt.jsの規約上で開発が可能で、プロジェクトをよく知らない人でも、なんとなくフォルダ構成がわかる
* SSR時に呼び出すかどうかを簡単に切り替えることができる。
    * SSRは通常のブラウザレンダリングではないため、JavaScriptでよくある`windowオブジェクト`や`document`などのDOMに関する操作はできない。したがって、プラグインの中で、`window`や`document`を参照すると、SSR時にエラーが発生する。
        * これに関しては、Node.jsの`process.browser`を利用し回避するという策もある。
    * プラグイン化することで、`nuxt.config.js上で`簡単に切り替えることができる。


### プラグインの実装と登録

プラグインを実装するには、`plugins`ディレクトリを作成し、その配下にjsファイルを作成する。
* Ex.`plugins/axios.js`
```js
import axios from 'axios';

export default ({ app }, inject) => {
    app.$axios = axios.create({
        baseURL: 'https://nuxt-blog-1c3e3.firebaseio.com'
    });
    inject('axios', app.$axios);
}
```
プラグインの登録は、`nuxt.config.js`に以下のように記述する。
```js
module.exports = {
    // ...
    plugins: [
        { src: '~/plugins/axios.js', ssr: true }
        //または、 以下のように、配列要素のみでも記述可能
        //'~/plugins/axios.js'
    ]
}
```

windowオブジェクトやdocumentなどのDOMに関する操作を行う場合は、`ssr: false`とする。
* SPAモードの時は、デフォルトで`ssr: false`となるので、明示的に記述する必要はない。


## 3.23 Vuexのモジュールモードを活用したオートローディング

Vuexには、クラシックモードとモジュールモードの２つがある。

|モード|説明|
|:--|:--|
|クラシックモード|自身でVuexを読み込み、ストアインスタンスを生成することでVuexストアを構築|
|モジュールモード|ストア自体に関わるコードは一切記載しない、ストアのモジュールロジックの記載を行うだけで構築.インスタンス生成はNuxt.js側で行う。|

### モジュールモードの仕組みについて


